# Test: the impact of extinction
---
Because extinction reduces the overall light, if it is not taken into account during the fit, then the SFH estimated by ppxf (particularly the stellar mass) may be inaccurate.
* Test 1: extincted vs. non-extincted WITHOUT mpoly in ppxf: how does it fare?
* Test 2: extincted vs. non-extincted WITH mpoly in ppxf: what does the best-fit polynomial look like? How accurate is the recovered SFH?


In [2]:
%matplotlib widget

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [4]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
from itertools import product
import multiprocessing
import pandas as pd
import extinction

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, compute_mw_age, compute_lw_age, compute_cumulative_mass, compute_cumulative_light
from ppxftests.sfhutils import compute_mean_age, compute_mean_mass, compute_mean_sfr, compute_mean_1D_sfh
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted, ppxf_plot
from ppxftests.plot_ppxf_summary import plot_ppxf_summary

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"
data_path = "/priv/meggs3/u5708159/ppxftests/"


In [65]:
###############################################################################
# Load a realistic SFH
###############################################################################
isochrones = "Padova"
SNR = 100
z = 0
niters = 20
nthreads = 20

x_AGN_input=0.5
alpha_nu_input=1.0

gal = 10
sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_star_kms = load_sfh(gal, plotit=True)
M_tot = np.nansum(sfh_mw_input)

###############################################################################
# Create spectrum
###############################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)

spec_agn, spec_agn_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=True, x_AGN=x_AGN_input, alpha_nu=alpha_nu_input,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)

# Apply extinction
A_V = 1.0
A_vals = extinction.fm07(lambda_vals_A, a_v=A_V, unit="aa")
ext_factor = 10**(-0.4 * A_vals)

spec_ext = spec * ext_factor
spec_ext_err = spec_err * ext_factor
spec_agn_ext = spec_agn * ext_factor
spec_agn_ext_err = spec_agn_err * ext_factor

# Plot 
fig, axs = plt.subplots(nrows=3, figsize=(15, 7.5))
axs[0].plot(lambda_vals_A, spec, label="Stellar continuum - before applying extinction")
axs[0].plot(lambda_vals_A, spec_ext, label="Stellar continuum - after applying extinction")
axs[1].plot(lambda_vals_A, spec_agn, label="Stellar continuum with AGN - before applying extinction")
axs[1].plot(lambda_vals_A, spec_agn_ext, label="Stellar continuum with AGN - after applying extinction")
axs[2].plot(lambda_vals_A, ext_factor, label="Extinction factor")
[ax.legend() for ax in axs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/ppxf_plot.py:113: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 3.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
###########################################################################
# Helper function for running MC simulations
###########################################################################
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A, mdegree, reddening, fit_agn_cont = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1, mdegree=mdegree, reddening=reddening,
                  regularisation_method="none",  
                  fit_agn_cont=fit_agn_cont,
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


In [49]:
###########################################################################
# Helper function for running our tests
###########################################################################
def test_helper(spec, spec_err, mdegree, reddening, fit_agn_cont):
    # Input arguments
    seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
    args_list = [[s, spec, spec_err, lambda_vals_A, mdegree, reddening, fit_agn_cont] for s in seeds]

    # Run in parallel
    print(f"Running ppxf on {nthreads} threads...")
    t = time()
    with multiprocessing.Pool(nthreads) as pool:
        pp_mc_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
    print(f"Total time in ppxf: {time() - t:.2f} s")

    ###########################################################################
    # Run ppxf with regularisation
    ###########################################################################
    t = time()
    pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1, mdegree=mdegree, reddening=reddening,
                  regularisation_method="auto",
                  fit_agn_cont=fit_agn_cont,
                  isochrones=isochrones,
                  fit_gas=False, tie_balmer=True,
                  delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
                  plotit=False, savefigs=False, interactive_mode=False)
    print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

    ###########################################################################
    # Summary plots
    ###########################################################################
    plot_ppxf_summary(sfh_lw_input, sfh_mw_input, pp_regul, pp_mc_list, isochrones)

    # Check the quality of the fit
    ppxf_plot(pp_regul); plt.gcf().suptitle(f"Regularised fit; mdegree={mdegree}; reddening={reddening}; fit_agn_cont={fit_agn_cont}")
    ppxf_plot(pp_mc_list[0]); plt.gcf().suptitle(f"MC fit; mdegree={mdegree}; reddening={reddening}; fit_agn_cont={fit_agn_cont}")
    
    return pp_regul, pp_mc_list


### Spectrum: NO extinction; ppxf: NO mpoly fit

In [39]:
pp_regul, pp_mc_list = test_helper(spec=spec, spec_err=spec_err, mdegree=-1, reddening=None, fit_agn_cont=False)

Running ppxf on 20 threads...



Total time in ppxf: 46.21 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 8.21 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3885...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 27.32 s
Iteration 1: optimal regul = 50000.00; Δm = 2.89318e+11; Δregul = 2500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 2.244
----------------------------------------------------
STOPPING: Optimal regul value is >= 50000.0; using 50000.00 to produce the best fit
Total time in run_ppxf: 45.12 seconds


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<ppxf.ppxf.ppxf at 0x7fed589a9e10>,
  <ppxf.ppxf.ppxf at 0x7fed59d75cd0>])

In [85]:
pp_regul.regul

50000.0

### Spectrum: WITH extinction; ppxf: NO mpoly fit

In [15]:
pp_regul, pp_mc_list = test_helper(spec=spec_ext, spec_err=spec_ext_err, mdegree=-1, reddening=None, fit_agn_cont=False)

Running ppxf on 20 threads...



Total time in ppxf: 24.70 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.30 s
----------------------------------------------------
Iteration 1: Scaling noise by 9.0103...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 22.33 s
Iteration 1: optimal regul = 0.00; Δm = 0.00113064; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 16.40 s
Iteration 2: optimal regul = -1000.00; Δm = 0; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 12.09 s
Iteration 3: optimal regul = -1200.00; Δm = 0; Δregul = 20.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 

/home/u5708159/python/Modules/ppxftests/sfhutils.py:316: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:255: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'WITH extinction; NO multiplicative polynomial in fit (MC)')

### Spectrum: WITH extinction; ppxf: WITH mpoly fit

In [16]:
pp_regul, pp_mc_list = test_helper(spec=spec_ext, spec_err=spec_ext_err, mdegree=4, reddening=None, fit_agn_cont=False)

Running ppxf on 20 threads...



Total time in ppxf: 72.61 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 6.58 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3790...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 107.20 s
Iteration 1: optimal regul = 10000.00; Δm = 8.79927e+10; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 27.216
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 117.80 s
Iteration 2: optimal regul = 20000.00; Δm = 5.89922e+08; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 18.819
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 106.11 s
Iteration 3: optimal regul = 30000.00; Δm = 3.717e+08; Δregul = 500.00 (Δregul_min

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'WITH extinction; NO multiplicative polynomial in fit (MC)')

In [25]:
# Compare the extinction curve with the polynomial from ppxf 
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(lambda_vals_A, ext_factor, color="black", label=r"$10^{-0.4 A_V}$")
ax.plot(lambda_vals_A, ext_factor / np.nanmean(ext_factor), color="grey", label=r"$10^{-0.4 A_V}$ (normalised)")
for pp in pp_mc_list:
    ax.plot(pp.lam, pp.mpoly, alpha=0.2, color="pink", label="mpoly (MC)" if pp == pp_mc_list[0] else None)
ax.plot(pp_regul.lam, pp_regul.mpoly, color="maroon", label="mpoly (regularised)")
ax.legend()
ax.set_xlabel(r"$\lambda$ (Å)")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$ (Å)')

### Remarks 
---
* The shape of the multiplicative polynomial is pretty bang-on.
* The LW and MW age estimates are both pretty accurate.
* *But...* the *absolute value* of the polynomial is way off. This must be because the multiplicative polynomial is designed to correct for small calibration errors, and so assumes that the *total* flux is fairly accurate. From the ppxf documentation: **MDEGREE: degree of the *multiplicative* Legendre polynomial (with mean of 1) used to correct the continuum shape during the fit (default: 0)**
* This means that the total MASS estimate is quite off.

### Spectrum: WITH extinction; ppxf: WITH extinction keyword

In [28]:
pp_regul, pp_mc_list = test_helper(spec=spec_ext, spec_err=spec_ext_err, mdegree=-1, reddening=3.0, fit_agn_cont=False)

Running ppxf on 20 threads...



Total time in ppxf: 47.20 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.72 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3751...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 46.06 s
Iteration 1: optimal regul = 10000.00; Δm = 2.55497e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 44.697
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 102.78 s
Iteration 2: optimal regul = 20000.00; Δm = 8.26014e+08; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 33.189
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 118.42 s
Iteration 3: optimal regul = 30000.00; Δm = 1.92922e+08; Δregul = 500.00 (Δregul_mi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'WITH extinction; WITH extinction correction in fit (MC)')

In [30]:
# Compare the extinction curve with the polynomial from ppxf 
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(lambda_vals_A, ext_factor, color="black", label=r"$10^{-0.4 A_V}$")
# ax.plot(lambda_vals_A, ext_factor / np.nanmean(ext_factor), color="grey", label=r"$10^{-0.4 A_V}$ (normalised)")
for pp in pp_mc_list:
    ax.plot(pp.lam, pp.mpoly, alpha=0.2, color="pink", label="mpoly (MC)" if pp == pp_mc_list[0] else None)
ax.plot(pp_regul.lam, pp_regul.mpoly, color="maroon", label="mpoly (regularised)")
ax.legend()
ax.set_xlabel(r"$\lambda$ (Å)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$ (Å)')

### Spectrum: WITH extinction; ppxf: WITH mpoly fit; WITH AGN continuum
* does ppxf accurately recover the AGN continuum?

In [83]:
pp_mpoly_regul, pp_mpoly_mc_list = test_helper(spec=spec_agn_ext, spec_err=spec_agn_ext_err, mdegree=4, reddening=None, fit_agn_cont=True)

Running ppxf on 20 threads...



Total time in ppxf: 68.36 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 6.52 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3748...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 206.08 s
Iteration 1: optimal regul = 10000.00; Δm = 1.18237e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 62.308
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 133.02 s
Iteration 2: optimal regul = 20000.00; Δm = 5.97673e+08; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 59.312
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 180.80 s
Iteration 3: optimal regul = 29500.00; Δm = 6.62802e+08; Δregul = 500.00 (Δregul_m

Process ForkPoolWorker-1121:
Process ForkPoolWorker-1140:
Process ForkPoolWorker-1136:
Process ForkPoolWorker-1135:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/pkg/linux/anaconda-201911

KeyboardInterrupt: 

  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1471, in __init__
    params, perror = self.nonlinear_fit(start1, bounds, fixed, tied, clean)
Process ForkPoolWorker-1123:
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1558, in nonlinear_fit
    x_scale='jac', tied=tied, fixed=fixed)
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/u5708159/python/Modules/ppxf/capfit.py", line 162, in __init__
    J2 = self.fdjac(func,

  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
KeyboardInterrupt
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1471, in __init__
    params, perror = self.nonlinear_fit(start1, bounds, fixed, tied, clean)
  File "/home/u5708159/python/Modules/ppxftests/run_ppxf.py", line 199, in ppxf_helper
    quiet=True)
  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1558, in nonlinear_fit
    x_scale='jac', tied=tied, fixed=fixed)
  File "/home/u5708159/python/Modules/p

KeyboardInterrupt
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1015, in nnls_flags
    x = optimize.nnls(AA, b)[0]
  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1716, in linear_fit
    self.weights = bvls_solve(aa, bb, npoly)
  File "/home/u5708159/python/Modules/ppxf/capfit.py", line 205, in call
    resid = func(p, *self.args, **self.kwargs)
  File "/home/u5708159/python/Modules/ppxf/capfit.py", line 142, in __init__
    f2 = self.call(func, p2)
Traceback (most recent call last):
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/pkg/linux/anaconda-2019

  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1716, in linear_fit
    self.weights = bvls_solve(aa, bb, npoly)
  File "/home/u5708159/python/Modules/ppxftests/run_ppxf.py", line 199, in ppxf_helper
    quiet=True)
  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1015, in nnls_flags
    x = optimize.nnls(AA, b)[0]
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/u5708159/python/Modules/ppxf/capfit.py", line 162, in __init__
    J2 = self.fdjac(func, p2, f2)
  File "/home/u5708159/python/Modules/ppxf/capfit.py", line 192, in fdjac
    f1 = self.call(func, pars1)
  File "/home/u5708159/python/Modules/ppxftests/run_ppxf.py", line 199, in ppxf_helper
    quiet=True)
  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1558, in nonlinear_fit
    x_scale='jac', tied=tied, fixed=fixed)
  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1094, in bvls_

  File "/home/u5708159/python/Modules/ppxf/ppxf.py", line 1015, in nnls_flags
    x = optimize.nnls(AA, b)[0]
KeyboardInterrupt
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/scipy/optimize/nnls.py", line 61, in nnls
    x, rnorm, mode = _nnls.nnls(A, m, n, b, w, zz, index, maxiter)
KeyboardInterrupt


In [81]:
# Compare the extinction curve with the polynomial from ppxf 
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(lambda_vals_A, ext_factor, color="black", label=r"$10^{-0.4 A_V}$")
ax.plot(lambda_vals_A, ext_factor / np.nanmean(ext_factor), color="grey", label=r"$10^{-0.4 A_V}$ (normalised)")
for pp in pp_mpoly_mc_list:
    ax.plot(pp.lam, pp.mpoly, alpha=0.2, color="pink", label="mpoly (MC)" if pp == pp_mpoly_mc_list[0] else None)
ax.plot(pp_mpoly_regul.lam, pp_mpoly_regul.mpoly, color="maroon", label="mpoly (regularised)")
ax.legend()
ax.set_xlabel(r"$\lambda$ (Å)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$ (Å)')

In [82]:
# Recreate the AGN continuum fit by ppxf
# In this case, because the polynomial is applied to the WHOLE spectrum including any sky components, the best-fit "sky" component should look similar to the input
# AGN continuum BEFORE the extinction correction is applied.
lambda_norm_A = 4020
F_lambda_fit = np.zeros(len(lambda_vals_A))

fig, ax = plt.subplots(figsize=(10, 5))

# Sum up the contributions of each template from the regularised ppxf fit.
for aa, alpha_nu in enumerate(pp_mpoly_regul.alpha_nu_vals):
    alpha_lambda = 2 - alpha_nu
    w_ppxf_regul = pp_mpoly_regul.weights_agn[aa]  # Weight of this template 
    x_AGN_fit_regul = (1 / w_ppxf_regul - 1)**(-1)  # Weight converted into light fraction of stellar continuum @ 4020Å
    
    F_0 = x_AGN_fit_regul / lambda_norm_A**(-alpha_lambda)  # Compute constant
    F_lambda_aa = F_0 * lambda_vals_A**(-alpha_lambda)  # Compute this component
    F_lambda_fit += F_lambda_aa  # Add to total fitted AGN continuum
    
    # Plot each contribution.
    ax.plot(lambda_vals_A, F_lambda_aa, label=f"alpha_nu = {alpha_nu}, x_AGN = {x_AGN_fit_regul}")

# Plot the TOTAL AGN continuum from the ppxf fit.
ax.plot(lambda_vals_A, F_lambda_fit, label="Total ppxf fit")

# Now, evaluate the INPUT AGN continuum.
alpha_lambda_input = 2 - alpha_nu_input
F_0_input = x_AGN_input / lambda_norm_A**(-alpha_lambda_input)
F_lambda_input = F_0_input * lambda_vals_A**(-alpha_lambda_input)

# Apply the extinction correction to the INPUT AGN continuum.
# NOTE: after this, we need to re-normalise F_lambda_input_ext to be equal to x_AGN at 4020Å, 
# because both the stellar & AGN continua are affected in identical ways by the extinction. 
# i.e., F_AGN(4020Å) = x_AGN * F_*(4020Å) AFTER the extinction is applied.
F_lambda_input_ext = F_lambda_input * ext_factor  
F_lambda_input_ext *= x_AGN_input / F_lambda_input_ext[np.nanargmin(np.abs(lambda_vals_A - lambda_norm_A))]

ax.plot(lambda_vals_A, F_lambda_input, label="Input AGN continuum")
ax.plot(lambda_vals_A, F_lambda_input_ext, label="Extincted input AGN continuum")
ax.axvline(lambda_norm_A, color="k", ls="--")

ax.legend()
ax.set_xlabel("Wavelenghth (Å)")
ax.set_ylabel(r"$x_{\rm AGN}(\lambda)$")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$x_{\\rm AGN}(\\lambda)$')

### Spectrum: WITH extinction; ppxf: WITH reddening; WITH AGN continuum
* does ppxf accurately recover the AGN continuum?

In [75]:
pp_ext_regul, pp_ext_mc_list = test_helper(spec=spec_agn_ext, spec_err=spec_agn_ext_err, mdegree=-1, reddening=3.0, fit_agn_cont=True)

Running ppxf on 20 threads...



Total time in ppxf: 51.91 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.30 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3500...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 96.12 s
Iteration 1: optimal regul = 50000.00; Δm = 2.97696e+11; Δregul = 2500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 59.854
----------------------------------------------------
STOPPING: Optimal regul value is >= 50000.0; using 50000.00 to produce the best fit
Total time in run_ppxf: 101.87 seconds


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
# Compare the extinction curve with the polynomial from ppxf 
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(lambda_vals_A, ext_factor, color="black", label=r"$10^{-0.4 A_V}$")
ax.plot(lambda_vals_A, ext_factor / np.nanmean(ext_factor), color="grey", label=r"$10^{-0.4 A_V}$ (normalised)")
for pp in pp_ext_mc_list:
    ax.plot(pp.lam, pp.mpoly, alpha=0.2, color="pink", label="mpoly (MC)" if pp == pp_ext_mc_list[0] else None)
ax.plot(pp_ext_regul.lam, pp_ext_regul.mpoly, color="maroon", label="mpoly (regularised)")
ax.legend()
ax.set_xlabel(r"$\lambda$ (Å)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$ (Å)')

In [77]:
# Compare the input AGN continuum with the fitted one
w_ppxf_regul = pp_ext_regul.weights_agn
x_AGN_fit_vals_regul = (1 / w_ppxf_regul - 1)**(-1)


In [78]:
# Recreate the AGN continuum fit by ppxf
# In this case, because the extinction correction is NOT applied to the sky templates within ppxf, the best-fit "sky" spectrum should look similar to the input 
# AGN spectrum AFTER extinction has been applied.
lambda_norm_A = 4020
F_lambda_fit = np.zeros(len(lambda_vals_A))

fig, ax = plt.subplots(figsize=(10, 5))

# Sum up the contributions of each template from the regularised ppxf fit.
for aa, alpha_nu in enumerate(pp_ext_regul.alpha_nu_vals):
    alpha_lambda = 2 - alpha_nu
    w_ppxf_regul = pp_ext_regul.weights_agn[aa]  # Weight of this template 
    x_AGN_fit_regul = (1 / w_ppxf_regul - 1)**(-1)  # Weight converted into light fraction of stellar continuum @ 4020Å
    
    F_0 = x_AGN_fit_regul / lambda_norm_A**(-alpha_lambda)  # Compute constant
    F_lambda_aa = F_0 * lambda_vals_A**(-alpha_lambda)  # Compute this component
    F_lambda_fit += F_lambda_aa  # Add to total fitted AGN continuum
    
    # Plot each contribution.
    ax.plot(lambda_vals_A, F_lambda_aa, label=f"alpha_nu = {alpha_nu}, x_AGN = {x_AGN_fit_regul}")

# Plot the TOTAL AGN continuum from the ppxf fit.
ax.plot(lambda_vals_A, F_lambda_fit, label="Total ppxf fit")

# Now, evaluate the INPUT AGN continuum.
alpha_lambda_input = 2 - alpha_nu_input
F_0_input = x_AGN_input / lambda_norm_A**(-alpha_lambda_input)
F_lambda_input = F_0_input * lambda_vals_A**(-alpha_lambda_input)

# Apply the extinction correction to the INPUT AGN continuum.
# NOTE: after this, we need to re-normalise F_lambda_input_ext to be equal to x_AGN at 4020Å, 
# because both the stellar & AGN continua are affected in identical ways by the extinction. 
# i.e., F_AGN(4020Å) = x_AGN * F_*(4020Å) AFTER the extinction is applied.
F_lambda_input_ext = F_lambda_input * ext_factor  
F_lambda_input_ext *= x_AGN_input / F_lambda_input_ext[np.nanargmin(np.abs(lambda_vals_A - lambda_norm_A))]

ax.plot(lambda_vals_A, F_lambda_input, label="Input AGN continuum")
ax.plot(lambda_vals_A, F_lambda_input_ext, label="Extincted input AGN continuum")
ax.axvline(lambda_norm_A, color="k", ls="--")

ax.legend()
ax.set_xlabel("Wavelenghth (Å)")
ax.set_ylabel(r"$x_{\rm AGN}(\lambda)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$x_{\\rm AGN}(\\lambda)$')